In [1]:
import sys

import pandas as pd
import numpy as np
import numdifftools as ndt
import matplotlib.pyplot as plt
import matplotlib.cm as cm

from scipy.stats import gamma
from scipy.stats import norm
from scipy.stats import vonmises
from scipy.stats import multivariate_normal
from scipy.stats import gaussian_kde
from scipy.stats import circstd
from scipy.special import iv
from scipy.special import expit
from scipy.special import logit
from scipy.special import logsumexp
from scipy.optimize import minimize
from scipy.optimize import minimize_scalar
from scipy.optimize import LinearConstraint
from scipy.signal import convolve
from scipy.interpolate import interp1d
from scipy.linalg import block_diag

from itertools import product

from math import isclose

from copy import deepcopy

from datetime import datetime

import importlib

import time
import pickle
import random

import stoch_optimizor
import optimizor
import HHMM

from helper_funcs import log_delta_2_eta0
from helper_funcs import log_Gamma_2_eta
from helper_funcs import eta0_2_log_delta
from helper_funcs import eta_2_log_Gamma
from helper_funcs import logdotexp

In [2]:
# parse command-line args
max_time = 600*1
id = 1

method_partialEs = [("control",0.0),
                    ("BFGS",0.0),
                    ("CG",0.0),
                    ("GD",0.0),
                    ("SAGA",0.0),
                    ("SAGA",0.5),
                    ("SAGA",1.0),
                    ("SVRG",0.0),
                    ("SVRG",0.5),
                    ("SVRG",1.0)]

rand_seed = [0,1,2,3,4,5,6,7,8,9]

# set methods
for i,settings0 in enumerate(product(rand_seed,method_partialEs)):
    if i == id:
        settings = settings0
        break

rand_seed = settings[0]
method = settings[1][0]
partial_E = settings[1][1]

random.seed(rand_seed)
np.random.seed(rand_seed)

print("method: %s" % method)
print("partial E_step: %.1f" % partial_E)
print("random seed: %d" % rand_seed)
print("max time : %.3f hours" % (max_time/3600))

# select parameters for optimization
num_epochs = 1000
tol = 1e-16
grad_tol = 1e-16

step_sizes = {"EM"  : [None,None],
              "CG"  : [None,None],
              "BFGS": [None,None],
              "GD"  : [0.005,0.005],
              "SGD" : [0.005,0.005],
              "SAG" : [0.005,0.005],
              "SVRG": [0.005,0.005],
              "SAGA": [0.005,0.005]}

jump_every = 1

### checks on optimization parameters ###
if partial_E > 0 and method in ["EM","BFGS","Nelder-Mead","CG"]:
    raise("partial_E not consistent with method")

### features of data ###
features = {'diveDuration'     : {'f'           : 'normal',
                                   'lower_bound' : None,
                                   'upper_bound' : None,
                                   'share_coarse': True,
                                   'share_fine'  : False},
             'maxDepth'         : {'f'           : 'normal',
                                   'lower_bound' : np.array([-np.infty,np.log(5),np.log(5)]),
                                   'upper_bound' : np.array([np.log(20),np.infty,np.infty]),
                                   'share_coarse': True,
                                   'share_fine'  : False},
             'avg_bot_htv'      : {'f'           : 'normal',
                                   'lower_bound' : None,
                                   'upper_bound' : None,
                                   'share_coarse': True,
                                   'share_fine'  : False}}



### load in data ###

# load in data
df = pd.read_csv("../../dat/Final_Data.csv")

# only take whale I107
#whales = ["I145","I107"]
#df = df[df["ID"].isin(whales)]

# convert times
df["stime"] = pd.to_datetime(df["stime"])
df["etime"] = pd.to_datetime(df["etime"])

# force dives to be at least 2 seconds long
df = df[df["diveDuration"] > np.log(2.0)]

# replace -inf
df["max_bot_jp"][df["max_bot_jp"] == -np.infty] = np.NAN

df["broadDiveType"] = np.NAN #3  # unknown
df.loc[df["maxDepth"] > np.log(20),"broadDiveType"] = 1  # deep
df.loc[df["maxDepth"] < np.log(5),"broadDiveType"] = 0  # shallow

# populate a data object

data = []

initial_ts = [0]
final_ts = []

for t,row in enumerate(df.iterrows()):
    
    if t != 0 and df.iloc[t]["ID"] != df.iloc[t-1]["ID"]:
        final_ts.append(t-1)
        initial_ts.append(t)
     
    data.append({"diveDuration"     : row[1]["diveDuration"],
                 "maxDepth"         : row[1]["maxDepth"],
                 "avg_bot_htv"      : row[1]["avg_bot_htv"]})#,
                 #"avg_bot_abs_roll" : row[1]["avg_bot_abs_roll"]})
                 #"postDiveInt"      : np.log(row[1]["postDiveInt"])})
                 #"max_bot_jp"       : row[1]["max_bot_jp"]})#,
                 #"foraging"         : row[1]["foraging"],
                 #"scales"           : row[1]["scales"],
                 #"click.train"      : row[1]["click.train"],
                 #"whales"           : row[1]["whales"]})
                
final_ts.append(t)

initial_ts = np.array(initial_ts)
final_ts = np.array(final_ts)

method: BFGS
partial E_step: 0.0
random seed: 0
max time : 0.167 hours


In [3]:
Ts = [1e3,1e4,1e5]
Ks = [3,5,10]
ds = [1,4,10]
rand_seed = [0,1,2,3,4,5,6,7,8,9]

id = 100

method_partialEs = [("control",0.0),
                    ("BFGS",0.0),
                    ("CG",0.0),
                    ("GD",0.0),
                    ("SAGA",0.0),
                    ("SAGA",0.5),
                    ("SAGA",1.0),
                    ("SVRG",0.0),
                    ("SVRG",0.5),
                    ("SVRG",1.0)]

# set methods
settings_star = (1000.0, 3, 1, 0, ('BFGS', 0.0))
i_star = 10
for i,settings in enumerate(product(Ts,Ks,ds,rand_seed,method_partialEs)):
    if settings == settings_star:#i == i_star:
        print(settings)
        print(i)
        break

(1000.0, 3, 1, 0, ('BFGS', 0.0))
1


In [4]:
from numpy import array
importlib.reload(HHMM)
importlib.reload(optimizor)
importlib.reload(stoch_optimizor)
T = len(data)
K = [2,3]

# pick intial parameters
optim = stoch_optimizor.StochOptimizor(data,features,K)

optim.initial_ts = initial_ts
optim.final_ts = final_ts

for feature in features:
    optim.param_bounds[feature] = {}
    optim.param_bounds[feature]["mu"] = [-100,100]
    optim.param_bounds[feature]["log_sig"] = [-5,5]

if method == "control":
    optim.step_size = step_sizes["SAGA"]
    if not (step_sizes["SAGA"][0] is None):
        optim.L_theta = 1.0 / step_sizes["SAGA"][0]
        optim.L_eta = 1.0 / step_sizes["SAGA"][1]
else:
    optim.step_size = step_sizes[method]
    if not (step_sizes[method][0] is None):
        optim.L_theta = 1.0 / step_sizes[method][0]
        optim.L_eta = 1.0 / step_sizes[method][1]

# print initial parameters
print("initial theta:")
print(optim.theta)
print("")
print("initial eta0:")
print(optim.eta0)
print("")
print("initial eta:")
print(optim.eta)
print("")

optim.theta = [{'diveDuration': {'mu': array([2.93665147, 4.71055169, 4.98305885]), 
                                 'log_sig': array([-0.54630145, -0.1220301 , -0.85076233])}, 
                'maxDepth': {'mu': array([0.68915082, 1.12681522, 2.13726734]), 
                             'log_sig': array([-0.61305886,  0.88465578, -0.1159796 ])}, 
                'avg_bot_htv': {'mu': array([-1.94030359, -0.98937075, -2.20728431]), 
                                'log_sig': array([-0.57657836, -0.59263875, -0.94045475])}}, 
               {'diveDuration': {'mu': array([2.93665147, 4.71055169, 4.98305885]), 
                                 'log_sig': array([-0.54630145, -0.1220301 , -0.85076233])}, 
                'maxDepth': {'mu': array([0.68915082, 1.12681522, 2.13726734]), 
                             'log_sig': array([-0.61305886,  0.88465578, -0.1159796 ])}, 
                'avg_bot_htv': {'mu': array([-1.94030359, -0.98937075, -2.20728431]), 
                                'log_sig': array([-0.57657836, -0.59263875, -0.94045475])}}]
optim.eta = [array([[ 0.        , -5.56375836],
                    [-5.69695586,  0.        ]]),
             [array([[  0.        ,  -1.72271983,  -4.60588735],
                     [  5.65621117,   0.        , -14.09841172],
                     [132.32424276, -33.68535577,   0.        ]]),
              array([[  0.        , -76.62103969,  -3.01498243],
                     [ 58.97936668,   0.        ,  -6.40284949],
                     [  4.85275022, -28.07081821,   0.        ]])]]
optim.eta0 = [array([ 0.       , 57.1931815]),
             [array([ 0.        , 14.34455776, 22.88665213]),
              array([  0.        , -10.02607645,   1.54587799])]]

initial theta:
[{'diveDuration': {'mu': array([0.67661836, 4.13440441, 2.04519336]), 'log_sig': array([-0.04082564, -0.04082564, -0.04082564])}, 'maxDepth': {'mu': array([2.97234926, 1.86204898, 1.40978261]), 'log_sig': array([-0.2768075, -0.2768075, -0.2768075])}, 'avg_bot_htv': {'mu': array([-2.13875961, -1.71382255, -2.52223635]), 'log_sig': array([-0.42452349, -0.42452349, -0.42452349])}}, {'diveDuration': {'mu': array([0.67661836, 4.13440441, 2.04519336]), 'log_sig': array([-0.04082564, -0.04082564, -0.04082564])}, 'maxDepth': {'mu': array([2.97234926, 1.86204898, 1.40978261]), 'log_sig': array([-0.2768075, -0.2768075, -0.2768075])}, 'avg_bot_htv': {'mu': array([-2.13875961, -1.71382255, -2.52223635]), 'log_sig': array([-0.42452349, -0.42452349, -0.42452349])}}]

initial eta0:
[array([0.        , 0.24853063]), [array([ 0.        , -0.03124534, -0.13947396]), array([ 0.        , -0.19065598,  0.44863753])]]

initial eta:
[array([[ 0.        , -3.13339524],
       [-2.48429903,  0. 

In [5]:
optim.grad_theta

[{'diveDuration': {'mu': array([0., 0., 0.]), 'log_sig': array([0., 0., 0.])},
  'maxDepth': {'mu': array([0., 0., 0.]), 'log_sig': array([0., 0., 0.])},
  'avg_bot_htv': {'mu': array([0., 0., 0.]), 'log_sig': array([0., 0., 0.])}},
 {'diveDuration': {'mu': array([0., 0., 0.]), 'log_sig': array([0., 0., 0.])},
  'maxDepth': {'mu': array([0., 0., 0.]), 'log_sig': array([0., 0., 0.])},
  'avg_bot_htv': {'mu': array([0., 0., 0.]), 'log_sig': array([0., 0., 0.])}}]

In [6]:
optim.get_log_like()

-25466.57718041519

# check gradient accuracy

In [7]:
optim.initial_ts

array([   0,  355, 1204, 2733, 3296, 4557, 5040, 5473, 6364, 6837, 7128])

In [8]:
optim.E_step()

In [9]:
display(optim.grad_theta)
#display(optim.grad_theta_t[0])
#display(optim.grad_theta_t[10])

[{'diveDuration': {'mu': array([-6.09631928e+03, -3.78695187e+00, -4.24282533e+01]),
   'log_sig': array([4209.66534434,   -5.16159775,   22.64413817])},
  'maxDepth': {'mu': array([3163.44153191,  -23.80237109,   56.47630276]),
   'log_sig': array([-2819.37404846,  -146.93844241,   -25.58003572])},
  'avg_bot_htv': {'mu': array([ 449.83938861, -194.9142067 ,  -54.87714247]),
   'log_sig': array([1794.65799064,   96.54334076,  -11.18582711])}},
 {'diveDuration': {'mu': array([-6.09631928e+03, -3.78695187e+00, -4.24282533e+01]),
   'log_sig': array([4209.66534434,   -5.16159775,   22.64413817])},
  'maxDepth': {'mu': array([3163.44153191,  -23.80237109,   56.47630276]),
   'log_sig': array([-2819.37404846,  -146.93844241,   -25.58003572])},
  'avg_bot_htv': {'mu': array([ 449.83938861, -194.9142067 ,  -54.87714247]),
   'log_sig': array([1794.65799064,   96.54334076,  -11.18582711])}}]

In [10]:
x0 = optim.params_2_x()

# get log likelihood to find hessian indices
def get_ll(x):

    print(x-x0)

    optim.x_2_params(x)

    for t in range(optim.T):
        optim.update_alpha(t)

    optim.x_2_params(x0)

    return logsumexp(optim.log_alphas[-1])

# get indices where hessian nonnegative
jacobian = ndt.Jacobian(get_ll,step=0.00001)(x0)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1.e-05 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00
 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00
 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00
 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00
 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00]
[-1.e-05  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00
  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00
  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00
  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00
  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00
  0.e+00  0.e+00  0.e+00]
[0.e+00 1.e-05 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00
 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00

[0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00
 0.e+00 1.e-05 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00
 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00
 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00
 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00]
[ 0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00
  0.e+00  0.e+00 -1.e-05  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00
  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00
  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00
  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00
  0.e+00  0.e+00  0.e+00]
[0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00
 0.e+00 0.e+00 1.e-05 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00
 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00
 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+

[ 0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00
  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00
  0.e+00  0.e+00  0.e+00  0.e+00 -1.e-05  0.e+00  0.e+00  0.e+00  0.e+00
  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00
  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00
  0.e+00  0.e+00  0.e+00]
[0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00
 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00
 0.e+00 0.e+00 0.e+00 1.e-05 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00
 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00
 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00]
[ 0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00
  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00
  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00 -1.e-05  0.e+00  0.e+00  0.e+00
  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00

[0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00
 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00
 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00
 0.e+00 0.e+00 0.e+00 0.e+00 1.e-05 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00
 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00]
[ 0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00
  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00
  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00
  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00 -1.e-05  0.e+00
  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00
  0.e+00  0.e+00  0.e+00]
[0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00
 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00
 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00
 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 1.e-05 0.e+00 0.e+

[ 0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00
  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00
  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00
  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00
  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00
 -1.e-05  0.e+00  0.e+00]
[0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00
 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00
 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00
 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00
 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 1.e-05 0.e+00]
[ 0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00
  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00
  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00
  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00

In [11]:
display(jacobian)
display(optim.grad_params_2_xprime())

array([[-5.99030102e+03,  1.62058031e+01,  2.23397852e+01,
         4.28118884e+03, -3.92919355e+00,  4.75057490e+01,
         3.34740746e+03, -9.60756443e+00,  8.29558807e+01,
        -2.62279655e+03, -7.28312583e+01,  1.31636170e+00,
         5.22852917e+02, -6.16436137e+01,  2.01090027e+02,
         1.85847963e+03,  5.06319420e+01,  1.58566080e+02,
         0.00000000e+00,  1.22007192e+01,  1.64339885e+01,
         0.00000000e+00,  0.00000000e+00, -1.56861387e+02,
        -7.16678478e+00, -1.09843249e+01,  0.00000000e+00,
         1.09139364e-06,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.83119894e+02,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00, -1.88059237e+01,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  3.76261578e+00, -3.76261778e+00,
         0.00000000e+00,  5.94759149e-01, -2.41447742e+00]])

array([-6.09631928e+03, -3.78695187e+00, -4.24282533e+01,  4.20966534e+03,
       -5.16159775e+00,  2.26441382e+01,  3.16344153e+03, -2.38023711e+01,
        5.64763028e+01, -2.81937405e+03, -1.46938442e+02, -2.55800357e+01,
        4.49839389e+02, -1.94914207e+02, -5.48771425e+01,  1.79465799e+03,
        9.65433408e+01, -1.11858271e+01,  0.00000000e+00,  1.19392790e+02,
        9.98569955e+01,  0.00000000e+00,  0.00000000e+00, -2.24724406e+01,
        9.90975231e+01, -4.55701180e+00,  0.00000000e+00,  1.30115460e+00,
       -2.53511416e+01,  1.20650268e+01,  0.00000000e+00,  0.00000000e+00,
        2.33842261e+02, -3.92512483e+01, -3.76631269e+01,  0.00000000e+00,
        4.14171995e+00, -1.35591390e+01,  5.32601685e+00,  0.00000000e+00,
        0.00000000e+00, -3.61500016e+00,  0.00000000e+00,  2.69488299e+01,
       -1.10371631e+02,  0.00000000e+00,  7.35957924e+00, -9.16292954e+01])

# make sure getting likelihood doesn't change alpha, beta, or grads

In [12]:
t = 10

print(optim.log_alphas[t])
print(optim.log_betas[t])
print(optim.p_Xt[t])
print(optim.p_Xtm1_Xt[t])
print("")
print(optim.get_ll_keep_params())
print("")
print(optim.log_alphas[t])
print(optim.log_betas[t])
print(optim.p_Xt[t])
print(optim.p_Xtm1_Xt[t])

[-41.83678543 -39.14409309 -31.57136223 -34.62607329 -54.58272444
 -28.93911364]
[-25432.01503111 -25431.69107255 -25432.76794725 -25429.16701562
 -25431.04112828 -25431.18254721]
[3.45476745e-05 2.35021266e-03 1.29869076e-01 4.43418350e-03
 2.46240689e-02 8.38687911e-01]
[[2.19828602e-05 9.64689289e-04 9.89107771e-02 1.44869021e-05
  1.51213980e-04 1.81829018e-02]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [1.25648143e-05 1.38552338e-03 3.09582992e-02 4.41969660e-03
  2.44728550e-02 8.20505009e-01]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]

(-22304.243653936857, 8727.5529138084)

[-41.83678543 -39.14409309 -31.57136223 -34.62607329 -54.58272444
 -28.93911364]
[-25432.01503111 -25431.69107255 -

In [13]:
print(optim.get_ll_keep_params())

(-22304.243653936857, 8727.5529138084)


In [14]:
print(optim.log_alphas)
print(optim.log_betas)

[[-8.83491989e+01 -7.28845349e+01 -6.16789664e+01 -1.00082471e+01
  -1.89142173e+01 -4.67879881e+00]
 [-3.55134845e+01            -inf            -inf -6.93764760e+00
             -inf            -inf]
 [-3.68697522e+01            -inf            -inf -8.94775662e+00
             -inf            -inf]
 ...
 [-2.22975026e+04            -inf            -inf -2.22951429e+04
             -inf            -inf]
 [           -inf -2.23029180e+04 -2.23052457e+04            -inf
  -2.23183515e+04 -2.23028174e+04]
 [-2.23049388e+04            -inf            -inf -2.23049347e+04
             -inf            -inf]]
[[-2.54615466e+04 -2.54611779e+04 -2.54625049e+04 -2.54590065e+04
  -2.54608709e+04 -2.54610106e+04]
 [-2.54584915e+04 -2.54580996e+04 -2.54595801e+04 -2.54561660e+04
  -2.54580217e+04 -2.54581599e+04]
 [-2.54556567e+04 -2.54551059e+04 -2.54549098e+04 -2.54536206e+04
  -2.54542492e+04 -2.54545353e+04]
 ...
 [-8.17273710e+00 -7.40896999e+00 -6.82634081e+00 -8.71262549e+00
  -7.98430694e